In [10]:
import pandas as pd 
import numpy as np 
import seaborn 
import datetime

In [3]:
biden = pandas.read_csv("biden_tweets.csv")
trump = pandas.read_csv("trump_tweets.csv")
# want to do cluster analysis, based on created_at date and favourites / retweets recieved 
# cluster by morning / day / evening <- created_at represented with military time 

In [22]:
biden.head()
biden.dtypes
date = biden["created_at"]

date[0]
biden["date_test"] = pd.to_datetime(biden["created_at"], format = "%Y-%m-%d %H:%M:%S")

In [24]:
biden["date_test"]

0      2020-10-07 02:12:00
1      2020-10-07 01:30:00
2      2020-10-07 01:05:00
3      2020-10-07 00:46:00
4      2020-10-07 00:15:00
               ...        
3079   2019-10-16 03:24:14
3080   2019-10-16 03:21:44
3081   2019-10-16 03:13:06
3082   2019-10-16 02:47:51
3083   2019-10-16 02:35:02
Name: date_test, Length: 3084, dtype: datetime64[ns]